In [131]:
import numpy as np
import pandas as pd
import geopandas
import altair
import pydeck as pdk
from carto_auth import CartoAuth
from pydeck_carto import register_carto_layer, get_layer_credentials
from pydeck_carto.layer import MapType, CartoConnection
from pydeck_carto.styles import color_continuous

In [103]:
geo_dpnk = geopandas.read_file('./data_project3/dpnk_json.geojson')
geo_roads = geopandas.read_file('./data_project3/cykloopatreni_json.geojson')

In [104]:
geo_roads = geo_roads[["rok_realizace", "delka", "geometry"]]
# geo_roads['centroid'] = geo_roads['geometry'].to_crs(crs=3857).centroid # to metres
# geo_roads['centroid'] = geo_roads['centroid'].to_crs(crs=4326) # to degrees

In [57]:
# Function that extracts the 2d list of coordinates from an input geometry
def my_geom_coord_extractor(input_geom):
    if (input_geom is None) or (input_geom is np.nan):
        return []
    else:
        if input_geom.geom_type[:len('multi')].lower() == 'multi':
            full_coord_list = []
            for geom_part in input_geom.geoms:
                geom_part_2d_coords = [[coord[0],coord[1]] for coord in list(geom_part.coords)]
                full_coord_list.append(geom_part_2d_coords)
        else:
            full_coord_list = [[coord[0],coord[1]] for coord in list(input_geom.coords)]
        return full_coord_list

# Applying the coordinate list extractor to the dataframe
geo_roads['path'] = geo_roads['geometry'].apply(my_geom_coord_extractor)
geo_roads['color'] = geo_roads.apply(lambda x: (237, 28, 36), axis=1)
geo_roads.head()

,rok_realizace,delka,geometry,centroid,path,color
0,2021.0,149.770000,"MULTILINESTRING ((16.59897 49.19989, 16.60020 ...",POINT (16.59998 49.20003),"[[[16.598970533000056, 49.199888574000056], [1...","(237, 28, 36)"
1,2017.0,71.870003,"MULTILINESTRING ((16.61335 49.19660, 16.61274 ...",POINT (16.61304 49.19648),"[[[16.613349483000036, 49.19659754200006], [16...","(237, 28, 36)"
2,2022.0,53.349998,"MULTILINESTRING ((16.64647 49.19613, 16.64645 ...",POINT (16.64642 49.19636),"[[[16.646472369000037, 49.19612806500004], [16...","(237, 28, 36)"
3,2021.0,248.250000,"MULTILINESTRING ((16.59679 49.19958, 16.59631 ...",POINT (16.59513 49.19935),"[[[16.596792452000045, 49.19958265000008], [16...","(237, 28, 36)"
4,2015.0,81.139999,"MULTILINESTRING ((16.60728 49.19359, 16.60718 ...",POINT (16.60707 49.19391),"[[[16.607281595000075, 49.193585658000075], [1...","(237, 28, 36)"


In [48]:
geo_roads['lat'] = geo_roads['centroid'].x
geo_roads['lon'] = geo_roads['centroid'].y
geo_roads = geo_roads.dropna()
geo_roads = geo_roads[['lat', 'lon']]

KeyError: 'centroid'

In [124]:
# geo_roads['color'] = geo_roads.apply(lambda x: (237, 28, 36), axis=1)
# geo_roads.apply(lambda x: print(x.name%2), axis=1)
geo_roads['color'] = geo_roads.apply(lambda x: (237, 28, 36) if x.name%2 else (180, 0, 200, 140), axis=1)
# geo_roads.loc[:5, 'color'] = (180, 0, 200, 140)
geo_roads


,rok_realizace,delka,geometry,color
0,2021.0,149.770000,"MULTILINESTRING ((16.59897 49.19989, 16.60020 ...","(180, 0, 200, 140)"
1,2017.0,71.870003,"MULTILINESTRING ((16.61335 49.19660, 16.61274 ...","(237, 28, 36)"
2,2022.0,53.349998,"MULTILINESTRING ((16.64647 49.19613, 16.64645 ...","(180, 0, 200, 140)"
3,2021.0,248.250000,"MULTILINESTRING ((16.59679 49.19958, 16.59631 ...","(237, 28, 36)"
4,2015.0,81.139999,"MULTILINESTRING ((16.60728 49.19359, 16.60718 ...","(180, 0, 200, 140)"
...,...,...,...,...
594,2015.0,17.150000,"MULTILINESTRING ((16.60907 49.19665, 16.60931 ...","(180, 0, 200, 140)"
595,2014.0,130.990010,"MULTILINESTRING ((16.61700 49.19058, 16.61654 ...","(237, 28, 36)"
596,2014.0,153.330000,None,"(180, 0, 200, 140)"
597,2014.0,132.759990,None,"(237, 28, 36)"


In [147]:
layer2 = pdk.Layer(
    type="GeoJsonLayer",
    data=geo_roads,
    pickable=True,
    # get_fill_color=[255, 255, 255],
    get_line_color='color',
    # get_width=100,
    line_width_scale=20
)
initial_position = pdk.ViewState(latitude=49.196023157428, longitude=16.60988, zoom=11, pitch=0, bearing=0)
deck = pdk.Deck(layers=[layer2], initial_view_state=initial_position, tooltip={"text": "Built in {rok_realizace}"})
# deck.to_html("path_layer.html")
deck

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 49.196023157428,
    "longitude": 16.60988,
    "pitch": 0,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": [
        {
          "color": [
            180,
            0,
            200,
            140
          ],
          "delka": 149.77,
          "geometry": {
            "coordinates": [
              [
                [
                  16.598970533000056,
                  49.199888574000056
                ],
                [
                  16.600197079000054,
                  49.200058959000046
                ],
                [
                  16.60055236200003,
                  49.20010830500007
                ],
                [
                  16.600790249000056,
                  49.200141365000036
                ],
                [
                  16.60098021500005,
                  49.200170393000064
                ]
              ]
            ],
            "type": "MultiLineString"
          },
          "rok_realizace": 2021.0
        },
        {
          "color": [
            237,
            28,
            36
          ],
          "delka": 71.870003,
          "geometry": {
            "coordinates": [
              [
                [
                  16.613349483000036,
                  49.19659754200006
                ],
                [
                  16.612735616000066,
                  49.19636295500004
                ]
              ]
            ],
            "type": "MultiLineString"
          },
          "rok_realizace": 2017.0
        },
        {
          "color": [
            180,
            0,
            200,
            140
          ],
          "delka": 53.349998,
          "geometry": {
            "coordinates": [
              [
                [
                  16.646472369000037,
                  49.19612806500004
                ],
                [
                  16.64645396800006,
                  49.19615860400006
                ],
                [
                  16.646426564000024,
                  49.19629188700003
                ],
                [
                  16.646399428000052,
                  49.196467245000065
                ],
                [
                  16.646388009000077,
                  49.196603193000044
                ]
              ]
            ],
            "type": "MultiLineString"
          },
          "rok_realizace": 2022.0
        },
        {
          "color": [
            237,
            28,
            36
          ],
          "delka": 248.25,
          "geometry": {
            "coordinates": [
              [
                [
                  16.596792452000045,
                  49.19958265000008
                ],
                [
                  16.596313958000053,
                  49.19951550600007
                ],
                [
                  16.59604672000006,
                  49.199478002000035
                ],
                [
                  16.59596321500004,
                  49.19946628800005
                ],
                [
                  16.595835328000078,
                  49.19944834900008
                ],
                [
                  16.595654703000037,
                  49.19942296000005
                ],
                [
                  16.59547259200002,
                  49.19939737400006
                ],
                [
                  16.595226129000025,
                  49.199362790000066
                ],
                [
                  16.594940545000043,
                  49.19932271500005
                ],
                [
                  16.59482741100004,
                  49.19930654600006
                ],
                [
                  16.59438271500005,
                  49.199242877000074
                ],
                [
                  16.593848876000038,
            

In [53]:
layer = pdk.Layer(
    'ScatterplotLayer',
    data=geo_roads,
    get_position=['lat', 'lon'],
    # get_position="[lat, lon]",
    auto_highlight=True,
    get_radius=50,          # Radius is given in meters
    get_fill_color=[180, 0, 200, 140],  # Set an RGBA value for fill
    pickable=True)

initial_position = pdk.ViewState(latitude=49.196023157428, longitude=16.60988, zoom=11, pitch=0, bearing=0)
deck = pdk.Deck(layers=[layer], initial_view_state=initial_position)
deck

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 49.196023157428,
    "longitude": 16.60988,
    "pitch": 0,
    "zoom": 11
  },
  "layers": [
    {
      "@@type": "ScatterplotLayer",
      "autoHighlight": true,
      "data": [
        {
          "lat": 16.599975764518856,
          "lon": 49.20002834031343
        },
        {
          "lat": 16.61304254950005,
          "lon": 49.19648024863912
        },
        {
          "lat": 16.64641860211663,
          "lon": 49.19636431993097
        },
        {
          "lat": 16.595127098324074,
          "lon": 49.19934821415284
        },
        {
          "lat": 16.607068753998004,
          "lon": 49.19391376471277
        },
        {
          "lat": 16.61620762379396,
          "lon": 49.18816662314377
        },
        {
          "lat": 16.58952581480983,
          "lon": 49.19596033613561
        },
        {
          "lat": 16.531293341652987,
          "lon": 49.19053767380198
        },
        {
          "lat": 16.610760816118155,
          "lon": 49.197056734439485
        },
        {
          "lat": 16.609591205000044,
          "lon": 49.199516503532884
        },
        {
          "lat": 16.591341542884084,
          "lon": 49.19647794791438
        },
        {
          "lat": 16.51888472746005,
          "lon": 49.21837349125661
        },
        {
          "lat": 16.49388192567298,
          "lon": 49.21881262040445
        },
        {
          "lat": 16.5846363498234,
          "lon": 49.214628284003325
        },
        {
          "lat": 16.630578357908874,
          "lon": 49.2283239073877
        },
        {
          "lat": 16.61222450650004,
          "lon": 49.20388844800258
        },
        {
          "lat": 16.58704840415521,
          "lon": 49.207257134447346
        },
        {
          "lat": 16.51245336111113,
          "lon": 49.206216070875776
        },
        {
          "lat": 16.612562232000048,
          "lon": 49.19232727010351
        },
        {
          "lat": 16.575676297809906,
          "lon": 49.1920190371041
        },
        {
          "lat": 16.600047560156604,
          "lon": 49.200039995003884
        },
        {
          "lat": 16.59735689441612,
          "lon": 49.19968674438654
        },
        {
          "lat": 16.61218594050004,
          "lon": 49.19622164150151
        },
        {
          "lat": 16.62060582245317,
          "lon": 49.18403415692391
        },
        {
          "lat": 16.648345745716274,
          "lon": 49.22106265543832
        },
        {
          "lat": 16.549074926458058,
          "lon": 49.23536467161143
        },
        {
          "lat": 16.52129722096957,
          "lon": 49.22224575028979
        },
        {
          "lat": 16.5871400653736,
          "lon": 49.20590272729495
        },
        {
          "lat": 16.660259862567578,
          "lon": 49.17884832144205
        },
        {
          "lat": 16.55685065623416,
          "lon": 49.21492583263495
        },
        {
          "lat": 16.483953179084217,
          "lon": 49.230239057295364
        },
        {
          "lat": 16.567597848191543,
          "lon": 49.210832101906085
        },
        {
          "lat": 16.60974117162048,
          "lon": 49.195547518916754
        },
        {
          "lat": 16.599916619948246,
          "lon": 49.20001999361682
        },
        {
          "lat": 16.59476343500006,
          "lon": 49.19992169199172
        },
        {
          "lat": 16.588685900060597,
          "lon": 49.22042528027422
        },
        {
          "lat": 16.613480720500064,
          "lon": 49.19664871202332
        },
        {
          "lat": 16.58864180300003,
          "lon": 49.20786704208005
        },
        {
          "lat": 16.587541938000072,
          "lon": 49.22012417463029
        },
        {
          "lat": 16.612255389571096,
          "lon": 49.19749059701921
        },
        {
          "lat": 16.604656165763583,
      